# Data Wrangling with MongoDB

### OpenStreetMap Sample Project

#### Map Area: Honolulu, Hawaii, United States


In [103]:
#importing all necessary files
import os
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import codecs
import json
import string
from pymongo import MongoClient

In [107]:
datadir = ""
datafile = "honolulu_hawaii.osm"
map_data = os.path.join(datadir, datafile)

##  Problems Encountered in the Map(Auditing)

In [120]:
def count_tags(filename): ##iterative parsing using Element tree to process the map file and find out what tags are there
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags
map_tags = count_tags(map_data)
pprint.pprint(map_tags)

{'bounds': 1,
 'member': 1612,
 'nd': 288004,
 'node': 242190,
 'osm': 1,
 'relation': 384,
 'tag': 117607,
 'way': 25500}


Here we founded that there are 242190 nodes, 117607 tags, 25500 ways and 384 relation.

In [121]:
#people invovlved in the map editing.
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
users = process_map(map_data)
len(users)

464

In [122]:
lower = re.compile(r'^([a-z]|_)*$')                    # Regular Expressions for different cases matching
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# initial expected street names
expected = ["Street", "Avenue", "Boulevard","Broadway","Drive", "Court", "Place", "Square", "Lane",
            "Road", "Trail", "Parkway"]

def key_type(element, keys):                  #Finding counts for different tag categories
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
                print tag.get('k')
            else:
                keys['other'] += 1
    return keys


def process_map(filename):                      #Distributing in different categories and finding tag count
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}  
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

map_keys = process_map(map_data)
pprint.pprint(map_keys)

{'lower': 64324, 'lower_colon': 50719, 'other': 2564, 'problemchars': 0}




    For the function: key_type & process_map. We check the "k" value for each "

    For the function 'key_type', we have a count of each of three tag categories in a dictionary: "lower", for tags that contain only lowercase letters and are valid, "lower_colon", for otherwise valid tags with a colon in their names, "problemchars", for tags with problematic characters, and



### Auditing Problems associating with street name abbreviations

In [123]:
def audit_street_type(street_types, street_name):    # add unexpected street name to a list
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    # determine whether a element is a street name
    return (elem.attrib['k'] == "addr:street")

def audit_street(osmfile):
    # iter through all street name tag under node or way and audit the street name value
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

st_types = audit_street(map_data)
# print out unexpected street names
pprint.pprint(dict(st_types))

{'106': set(['Pualei Cir, Apt 106']),
 'Ave': set(['Kalakaua Ave']),
 'Blvd': set(['Ala Moana Blvd']),
 'Center': set(['Enchanted Lakes Shopping Center']),
 'Circle': set(['Delay Circle', 'Papu Circle', 'Pualei Circle']),
 'Dr': set(['Kipapa Dr']),
 'Highway': set(['Farrington Highway',
                 "Kalaniana'ole Highway",
                 'Kalanianaole Highway',
                 u'Kalaniana\u2019ole Highway',
                 'Kamehameha Highway',
                 'Nimitz Highway',
                 'Pali Highway']),
 'Honolulu': set(['Moanalua, Honolulu']),
 'Hwy': set(['Kamehameha Hwy']),
 'Ike': set(['Ala Ike']),
 'Kailua,': set(['Kaelepulu Dr, Kailua,']),
 'King': set(['South King']),
 'Loop': set(['98-402 Koauka Loop',
              '98-410 Koauka Loop',
              '98-500 Koauka Loop',
              '98-501 Koauka Loop']),
 'Mall': set(['Fort Street Mall', 'McCarthy Mall']),
 'Momi': set(['Pali Momi']),
 'Pkwy': set(['Meheula Pkwy']),
 'St': set(['Ala Pumalu St', 'Lusitan

In [146]:
mapping={  'Ave'  : 'Avenue',             #mapping abbreviations to their full form
           'Ave.' : 'Avenue',
           'Apt'  : 'Apartment',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'  : 'Road',
           'St'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'St.'  : 'Street',
           'Sq'   : "Square",
           '420'  : "420",
           'Ext'  : "Extension",
        }

In [147]:
def update_name(name, mapping, regex):                         #updating street names to correct form
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

for street_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping, street_type_re)
        print name, "=>", better_name


Ala Ike => Ala Ike
Lusitania St. => Lusitania Street
Pualani Way => Pualani Way
Wai Nani Way => Wai Nani Way
Kuaaina Way => Kuaaina Way
Ainakea Way => Ainakea Way
Coelho Way => Coelho Way
Papu Circle => Papu Circle
Pualei Circle => Pualei Circle
Delay Circle => Delay Circle
Pali Highway => Pali Highway
Farrington Highway => Farrington Highway
Kamehameha Highway => Kamehameha Highway
Kalaniana’ole Highway => Kalaniana’ole Highway
Kalanianaole Highway => Kalanianaole Highway
Kalaniana'ole Highway => Kalaniana'ole Highway
Nimitz Highway => Nimitz Highway
Pali Momi => Pali Momi
Moanalua, Honolulu => Moanalua, Honolulu
Kamehameha Hwy => Kamehameha Highway
Kaelepulu Dr, Kailua, => Kaelepulu Dr, Kailua,
Kipapa Dr => Kipapa Drive
kanehameha highway => kanehameha highway
South King => South King
Enchanted Lakes Shopping Center => Enchanted Lakes Shopping Center
Meheula Pkwy => Meheula Parkway
Fort Street Mall => Fort Street Mall
McCarthy Mall => McCarthy Mall
Ala Pumalu St => Ala Pumalu Street


### Problems in zip codes

In [148]:
from collections import defaultdict     # Auditing all zip codes in the given data

def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if not zipcode.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
    
    elif twoDigits !="96" :
        invalid_zipcodes[twoDigits].add(zipcode) 
    elif len(zipcode)>5:
        invalid_zipcodes[twoDigits].add(zipcode) 
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

map_zipcode = audit_zip(map_data)
map_zipcode

defaultdict(set,
            {'96': {'96712-9998',
              '96734-9998',
              '96815-2518',
              '96815-2830',
              '96815-2834',
              '96817-1713',
              '96825-9998',
              '96826-4427'},
             'HI': {'HI 96819'}})

Some zip codes contain hypen b/w them.

One type of zip code contain text.

In [149]:
def update_zip(zipcode):                          # Fixing zip codes
    testNum = re.findall('[a-zA-Z]*', zipcode)
    if testNum:
        testNum = testNum[0]
    testNum.strip('-')
    if testNum == "HI":
        convertedZipcode = (re.findall(r'\d+', zipcode))
        if convertedZipcode:
            if convertedZipcode.__len__() == 2:
                return (re.findall(r'\d+', zipcode))[0] + "-" +(re.findall(r'\d+', zipcode))[1]
            else:
                return (re.findall(r'\d+', zipcode))[0]
    else:        
        return (re.findall(r'\d+', zipcode))[0]
for street_type, ways in map_zipcode.iteritems():
    for name in ways:
        better_name = update_zip(name)
        print name, "=>", better_name

HI 96819 => 96819
96815-2518 => 96815
96734-9998 => 96734
96826-4427 => 96826
96817-1713 => 96817
96815-2830 => 96815
96815-2834 => 96815
96712-9998 => 96712
96825-9998 => 96825


Zip code errors were solved i.e removing hypen and text characters.

## Formatting data in required json Format

In [150]:
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]    #created field for json object
# function that corrects incorrect street names
def update_name(name, mapping):    
    for key in mapping:
        if key in name:
            name = string.replace(name,key,mapping[key])
    return name

def shape_element(element):
    node = {}
    node["created"]={}
    node["address"]={}
    node["pos"]=[]
    refs=[]
    
    # we only process the node and way tags
    if element.tag == "node" or element.tag == "way" :
        if "id" in element.attrib:
            node["id"]=element.attrib["id"]
        node["type"]=element.tag

        if "visible" in element.attrib.keys():
            node["visible"]=element.attrib["visible"]
      
        # the key-value pairs with attributes in the CREATED list are added under key "created"
        for elem in CREATED:
            if elem in element.attrib:
                node["created"][elem]=element.attrib[elem]
                
        # attributes for latitude and longitude are added to a "pos" array
        # include latitude value        
        if "lat" in element.attrib:
            node["pos"].append(float(element.attrib["lat"]))
        # include longitude value    
        if "lon" in element.attrib:
            node["pos"].append(float(element.attrib["lon"]))

        
        for tag in element.iter("tag"):
            if not(problemchars.search(tag.attrib['k'])):
                if tag.attrib['k'] == "addr:housenumber":
                    node["address"]["housenumber"]=tag.attrib['v']
                    
                if tag.attrib['k'] == "addr:postcode":
                    node["address"]["postcode"]=tag.attrib['v']
                
                # handling the street attribute, update incorrect names using the strategy developed before   
                if tag.attrib['k'] == "addr:street":
                    node["address"]["street"]=tag.attrib['v']
                    node["address"]["street"] = update_name(node["address"]["street"], mapping)

                if tag.attrib['k'].find("addr")==-1:
                    node[tag.attrib['k']]=tag.attrib['v']
                    
        for nd in element.iter("nd"):
             refs.append(nd.attrib["ref"])
                
        if node["address"] =={}:
            node.pop("address", None)

        if refs != []:
           node["node_refs"]=refs
            
        return node
    else:
        return None

# process the xml openstreetmap file, write a json out file and return a list of dictionaries
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [151]:
# process the file
data = process_map(map_data, True)

## Storing processed json data to MongoDB

In [159]:
'''client = MongoClient()      #importing json data to mongo db locally
db = client.honolulu
collection = db.honolulu
collection.insert(data)'''

'client = MongoClient()      #importing json data to mongo db locally\ndb = client.honolulu\ncollection = db.honolulu\ncollection.insert(data)'

In [160]:
collection         #collection named honolulu for this data

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'honolulu'), u'honolulu')

## Data Overview with MongoDB

In [161]:
os.path.getsize(cal_data)/1024/1024        #Size of xml file in MB

50

In [162]:
os.path.getsize("honolulu_hawaii.osm.json")/1024/1024      #Size Of json file in MB

72

In [163]:
collection.find().count()      #Total number of documents inside Mongo DB collection honolulu

267690

In [164]:
# Number of unique users
len(collection.group(["created.uid"], {}, {"count":0}, "function(o, p){p.count++}"))


461

In [165]:
# Number of nodes
collection.find({"type":"node"}).count()

242159

In [166]:
# Number of ways
collection.find({"type":"way"}).count()

25495

In [168]:
#Top five users with most contributions
pipeline = [{"$group":{"_id": "$created.user",     
                       "count": {"$sum": 1}}},
            {"$sort": {"count": -1}},
            {"$limit": 5}]
result = collection.aggregate(pipeline)
x=list(result)
x


[{u'_id': u'Tom_Holland', u'count': 102211},
 {u'_id': u'cbbaze', u'count': 14995},
 {u'_id': u'ikiya', u'count': 12807},
 {u'_id': u'kr4z33', u'count': 9470},
 {u'_id': u'Chris Lawrence', u'count': 9214}]

this shows that Tom_Holland is most active user.

In [169]:
#Proportion of the top user contributions
top_user_prop = [{"$group":{"_id": "$created.user",
                       "count": {"$sum": 1}}},
            {"$project": {"proportion": {"$divide" :["$count",collection.find().count()]}}},
            {"$sort": {"proportion": -1}},
            {"$limit": 5}]
result = list(collection.aggregate(top_user_prop))
result

[{u'_id': u'Tom_Holland', u'proportion': 0.3818259927528111},
 {u'_id': u'cbbaze', u'proportion': 0.05601628749673129},
 {u'_id': u'ikiya', u'proportion': 0.04784265381598117},
 {u'_id': u'kr4z33', u'proportion': 0.03537674175352087},
 {u'_id': u'Chris Lawrence', u'proportion': 0.03442041167021555}]

38% contribution was alone by Tom_Holland and others have very less compared to him.

## Additional Ideas and further exploration

In [173]:
#Most common amenities in the area
aminity=[{"$match":{"amenity":{"$exists":1}}},
                              {"$group":{"_id":"$amenity","count":{"$sum":1}}},
                              {"$sort":{"count":-1}}, {"$limit":10}]
result=list(collection.aggregate(aminity))
result

[{u'_id': u'parking', u'count': 391},
 {u'_id': u'restaurant', u'count': 218},
 {u'_id': u'fast_food', u'count': 112},
 {u'_id': u'school', u'count': 85},
 {u'_id': u'toilets', u'count': 75},
 {u'_id': u'cafe', u'count': 64},
 {u'_id': u'place_of_worship', u'count': 38},
 {u'_id': u'library', u'count': 38},
 {u'_id': u'fire_station', u'count': 36},
 {u'_id': u'college', u'count': 29}]

Most frequent amenities are parking, restaurant, fast_food respectively.

In [172]:
# Types of parking and their frequency 
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"parking", "parking":{"$exists":1}}}, 
            {"$group":{"_id":"$parking", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":10}]
result = list(collection.aggregate(pipeline))
result

[{u'_id': u'surface', u'count': 135},
 {u'_id': u'multi-storey', u'count': 54},
 {u'_id': u'underground', u'count': 3}]

Surface Parking is mostly performed so good road parking facilities in the area

In [174]:
#cuisines in restaurants
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant", "cuisine":{"$exists":1}}}, 
            {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":10}]
result = list(collection.aggregate(pipeline))
result

[{u'_id': u'pizza', u'count': 8},
 {u'_id': u'japanese', u'count': 7},
 {u'_id': u'regional', u'count': 6},
 {u'_id': u'chinese', u'count': 5},
 {u'_id': u'american', u'count': 5},
 {u'_id': u'international', u'count': 4},
 {u'_id': u'thai', u'count': 4},
 {u'_id': u'italian', u'count': 3},
 {u'_id': u'asian', u'count': 3},
 {u'_id': u'indian', u'count': 2}]

In [175]:
#cuisines in fast_food
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"fast_food", "cuisine":{"$exists":1}}}, 
            {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":10}]
result = list(collection.aggregate(pipeline))
result

[{u'_id': u'burger', u'count': 18},
 {u'_id': u'sandwich', u'count': 5},
 {u'_id': u'mexican', u'count': 4},
 {u'_id': u'pizza', u'count': 3},
 {u'_id': u'sushi', u'count': 3},
 {u'_id': u'ice_cream', u'count': 1},
 {u'_id': u'asian', u'count': 1},
 {u'_id': u'american', u'count': 1},
 {u'_id': u'hawaiian', u'count': 1},
 {u'_id': u'regional', u'count': 1}]

burger is most popular cuisine in fast_food

In [178]:
#name of different schools
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "school", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}},{"$limit":10}]
result = list(collection.aggregate(pipeline))
result


[{u'_id': u"Kawaiaha'o Church School", u'count': 1},
 {u'_id': u'The Kamehameha Schools', u'count': 1},
 {u'_id': u'Noelani School', u'count': 1},
 {u'_id': u"Ka'elepulu Elementary", u'count': 1},
 {u'_id': u'Hawaii School for the Deaf and the Blind', u'count': 1},
 {u'_id': u'Mililani Ike Elementary School', u'count': 1},
 {u'_id': u'R. L. Stevenson Middle School', u'count': 1},
 {u'_id': u'Shade House', u'count': 1},
 {u'_id': u'Highlands Intermediate School', u'count': 1},
 {u'_id': u'Mid-Pacific Institute', u'count': 1}]

There are only single branch of every school in this city.

In [180]:
# types of roads or highways
pipeline = [{'$match': {'highway': { '$exists': 1}}}, 
        {'$group': {'_id': '$highway',
                    'count': {'$sum': 1}}}, 
        {'$sort': {'count': -1}},
        {'$limit': 5}]
result=list(collection.aggregate(pipeline)) 
result

[{u'_id': u'residential', u'count': 6832},
 {u'_id': u'service', u'count': 4538},
 {u'_id': u'living_street', u'count': 1759},
 {u'_id': u'turning_circle', u'count': 1354},
 {u'_id': u'footway', u'count': 555}]

So mostly residential area is there in the city which highway covers

In [181]:
# types of religion followed
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
                      {"$group":{"_id":"$religion", "count":{"$sum":1}}},
                      {"$sort":{"count":-1}}]
result=list(collection.aggregate(pipeline)) 
result

[{u'_id': u'christian', u'count': 22},
 {u'_id': None, u'count': 9},
 {u'_id': u'buddhist', u'count': 6},
 {u'_id': u'muslim', u'count': 1}]

So mostly Christian religion is followed ...

### Additional Idea

For improving the database some validations should be applied while filling the information in specific way.

And also user who contributed most should be displayed on the website so that others also try to contribute more by getting motivation from his/her work.

>From above two ideas firstly data is entered in more proper format.
Secondly contribution increases if you give some credit to persons who contributed more.

# Conclusion

From Data Wrangling on Honolulu, Hawai, USA openstreet data of size greater than 50 MB I found that:

464 people were involved in its map editing out of which 1 contributed the most around 38%.

Some errors in street name abbreviations and zip codes were found while auditing the dataset which were fixed by mapping.

Parking is most frequent amenity in the area and in parking surface parking is most common.

People there are fond of pizzas and burgers and other fast food items. 

Mostly Christians are there in that area.

So data must have lots of other hidden information as any type of tag is allowed and could be more further explored.

## Refrences

https://docs.google.com/document/d/1F0Vs14oNEs2idFJR3C_OPxwS6L0HPliOii-QpbmrMo4/pub#h.ueey7dly83g7

https://mapzen.com/data/metro-extracts/metro/honolulu_hawaii/

Case study of lectures

http://wiki.openstreetmap.org/wiki/